In [2]:
%load_ext rpy2.ipython
!rm -rf RiesgosFinancieros
!git clone "https://github.com/pikey-msc/RiesgosFinancieros/"



Cloning into 'RiesgosFinancieros'...
remote: Enumerating objects: 768, done.
remote: Counting objects: 100% (78/78), done.
remote: Compressing objects: 100% (38/38), done.
remote: Total 768 (delta 74), reused 40 (delta 40), pack-reused 690
Receiving objects: 100% (768/768), 41.13 MiB | 33.40 MiB/s, done.
Resolving deltas: 100% (509/509), done.
Checking out files: 100% (409/409), done.


In [3]:
%%R
#require(quantmod)
install.packages("quantmod")
library(quantmod)
#require(data.table)
install.packages("data.table")
library(data.table)
#require("PerformanceAnalytics")
install.packages("PerformanceAnalytics")
library("PerformanceAnalytics")

install.packages("Deriv")
library(Deriv)


library(dplyr)
con = gzcon(url('https://github.com/systematicinvestor/SIT/raw/master/sit.gz', 'rb'))
source(con)
close(con)

R[write to console]: Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

R[write to console]: also installing the dependencies ‘xts’, ‘zoo’, ‘TTR’


R[write to console]: trying URL 'https://cran.rstudio.com/src/contrib/xts_0.12.1.tar.gz'

R[write to console]: Content type 'application/x-gzip'
R[write to console]:  length 517875 bytes (505 KB)

R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[

In [4]:
%%R
#FUNCIÓN DE INTERPOLACIÓN ALAMBRADA

talamb=function(nodos,curva,plazos) #función de interpolación de tasas por el método alamabrada
{
  n=max(ncol(plazos),1)
  m=max(ncol(nodos),1)
  TC=matrix(0,1,n)
  TL=matrix(0,1,n)
  TF=matrix(0,1,n)
  for (j in 1:n)
  {
    i=1
    repeat
    {
      if(nodos[i]<= plazos[j] && plazos[j] <=nodos[i+1])
      {
        TC[j]=curva[i]
        TL[j]=curva[i+1]
        TF[j]=((((1+TL[j]*nodos[i+1]/360)/(1+TC[j]*nodos[i]/360))^((plazos[j]-nodos[i])/(nodos[i+1]-nodos[i]))*(1+TC[j]*nodos[i]/360))-1)*360/plazos[j]
        break
      }
      else if (plazos[j]<nodos[1])
      {
        TC[j]=curva[1]
        TL[j]=curva[1]
        TF[j]=curva[1]
        break
      }
      else if (plazos[j]>nodos[m])
      {
        TC[j]=curva[m]
        TL[j]=curva[m]
        TF[j]=curva[m]
        break
      }
      else
      {i=i+1}
    }
  }
  as.matrix(t(as.numeric(rbind(TF))))
}



# Definición de parámetros para valorar:
Futuros

Cargamos parámetros para valoración:

In [5]:
%%R
fval=as.Date("20220331",format="%Y%m%d") #Fecha de valoración
itpl=0 #poner 0 si se quiere interpolación lineal o 1 si se quiere tasa alambrada
alpha=0.98 #Nivel de confianza para obtener estimaciones de riesgo
nh=3660 #número de días de historia
yext=1 #si se usa la historia de internet o fija
#setwd(direc)	



#FUTUROS DE TDC
bext="RiesgosFinancieros/2020-2/Insumos/tasa_libor.txt" 
bdom="RiesgosFinancieros/2020-2/Insumos/tasa_fwd.txt"
btsp="RiesgosFinancieros/2020-2/Insumos/tasa_spot.txt"
#SymbolsFX_ftdc<-c("USDMXN=X", "GBPUSD=X" ) #tienen que ir en orden alfabético
SymbolsFX_ftdc<-"USDMXN=X"
plazos_fowd=cbind( 5,53	)
#plazos_fowd = 5
#contratos_fwd=cbind(100,50)
contratos_fowd = 100 
#kst_fwd=cbind(20.83,49525)
kst_fwd=20.83

nominal_fwd=1
yext=0 #si se carga información de yahoo en la fecha definida por fval o SymbolsFX, en caso contrario se utiliza información que se tendrá que cargar de tasas_spot.txt
trlib=1 #1 si la curva libor viene a 182 0 si no.

#FUTUROS DE IPC
#Descontamos con gubernamental
base="RiesgosFinancieros/2020-2/Insumos/tasa_guber.txt"
#SymbolsEQ_find<-c("^MXX", "GCARSOA1.MX" ) #tienen que ir en orden alfabético
SymbolsEQ_find<-"^MXX"
#plazos_fwd_ind=cbind( 360,	9,	3500)
plazos_fwd_ind= 53 
#contratos_fwd_ind=cbind(5, -2, 5)
contratos_fwd_ind= 50
#kst_fwd_ind=cbind(52000, 49500, 60000)
kst_fwd_ind=49525 
nominal_fwd_ind=1




In [6]:

%%R
# CARGA DE DATOS DE  FORWARD DE IPC
data3<-read.table(btsp)
print(head(data3))
n3<-nrow(data3)
m3<-ncol(data3)
X3=data.table(as.matrix(as.double(as.matrix(data3[2:(n3+1),m3]))))
X3_find=as.data.table(mutate(data3[2:n3+1,1:m3],Date=as.Date(V1,format="%Y%m%d")))
print(head(X3_find))


        V1      V2
1     date  precio
2 20200310 21.1117
3 20200309 20.1623
4 20200306 19.8095
5 20200305 19.5335
6 20200304 19.2837
         V1      V2       Date
1: 20200309 20.1623 2020-03-09
2: 20200306 19.8095 2020-03-06
3: 20200305 19.5335 2020-03-05
4: 20200304 19.2837 2020-03-04
5: 20200303 19.6985 2020-03-03
6: 20200302  19.776 2020-03-02


# Carga de datos

In [28]:
%%R


#CARGA DE DATOS DE  FORWARDS DE TDC
#datas
#data<-read.table("tasa_tiie.txt")
data1<-read.table(bext)
data2<-read.table(bdom)

########minimos para parametrizar
n1=nrow(data1)
n2=nrow(data2)
m1_ftdc=ncol(data1)
m2_ftdc=ncol(data2)
n=min(n1,n2)-1
###NODOS###
nodos1_ftdc=data.frame(data1[1,2:m1_ftdc])
nodos2_ftdc=data.frame(data2[1,2:m2_ftdc])

####MATRICES DEL MISMO TAMAÑO MENOS DOLAR

x1_ftdc=as.data.table(mutate(data1[2:n1,1:m1_ftdc],Date=as.Date(V1,format="%Y%m%d")))
x1_ftdc=x1_ftdc%>%select(-V1)
x2_ftdc=as.data.table(mutate(data2[2:n1,1:m2_ftdc],Date=as.Date(V1,format="%Y%m%d")))
x2_ftdc=x2_ftdc%>%select(-V1)

###Para Dolar

if (yext==1)
{
  #Cargar los símbolos de yahoo finance para FX
  start_date=fval-3660 #fecha inicial
  
  #Creación del objeto para guardar los datos
  dataEnvFX<-new.env()
  
  #obtener los datos
  getSymbols.yahoo(SymbolsFX_ftdc,env=dataEnvFX,from=start_date, to=(fval))
  #limpiarlos, alinearnos y quedarnos con el precio de cierre 
  bt.prep(dataEnvFX,align='remove.na',fill.gaps=T)
  
  #muestra de datos
  head(dataEnvFX$prices[,2])
  
  #Nos quedamos con los precios
  X3_ftdc=data.table(Date=as.Date(index(dataEnvFX$prices[,2])),coredata(dataEnvFX$prices[,2]))
} else  
{ 
data3<-read.table(btsp)
print(head(data3))
n3<-nrow(data3)
m3<-ncol(data3)
X3=data.table(as.matrix(as.double(as.matrix(data3[2:(n3+1),m3]))))
X3_find=as.data.table(mutate(data3[2:(n+1),1:m3],Date=as.Date(V1,format="%Y%m%d")))
}
# CARGA DE DATOS DE  FORWARD DE IPC

data1<-read.table(base)
n<-nrow(data1)
m_gov=ncol(data1)
#x_orig_gov=data.frame(data[2:n,1:m_gov])
x_orig_gov=as.data.table(mutate(data1[2:n,1:m_gov],Date=as.Date(V1,format="%Y%m%d")))
x_orig_gov=x_orig_gov%>%select(-V1)
nodos_gov=data.frame(data1[1,2:m_gov])

  #Cargar los símbolos de yahoo finance para EQ
  start_date=fval-nh #fecha inicial
  #Creación del objeto para guardar los datos
  dataEnvEQ<-new.env()
   
#obtener los datos
  getSymbols.yahoo(SymbolsEQ_find,env=dataEnvEQ,from=start_date, to=(fval))
  #limpiarlos, alinearnos y quedarnos con el precio de cierre 
  #bt.prep(dataEnvEQ,align='remove.na',fill.gaps= 1)
  
 #muestra de datos
#  head(dataEnvEQ$prices)
  
#Nos quedamos con los precios
  X3_find=data.table(Date=as.Date(index(dataEnvEQ$prices[,2])),coredata(dataEnvEQ$prices[,2]))



 


        V1      V2
1     date  precio
2 20200310 21.1117
3 20200309 20.1623
4 20200306 19.8095
5 20200305 19.5335
6 20200304 19.2837


# Integración de Insumos

Aseguramiento de que todos los factores de riesgo estén alineados en sus fechas, nos quedamos con la intersección. Posteriormente, filtramos cada histórico factor de riesgo hacia esta intersección de todos.

In [29]:
%%R
#INTERSECCIÓN DE FECHAS DE TODOS LOS INSUMOS    
  
head(x_orig_gov)  
lin_gub=data.table(Date=as.Date(X3_find[x_orig_gov,on=.(Date),nomatch=0]$Date)) #Fechas acciones, equity y guber
lin_gub_bmybdst_flib=data.table(Date=as.Date(lin_gub[x1_ftdc,on=.(Date),nomatch=0]$Date)) #Fechas acciones, equity, guber, st (bonde), libor
lin_gub_bmybdst_flibfwd=data.table(Date=as.Date(lin_gub_bmybdst_flib[x2_ftdc,on=.(Date),nomatch=0]$Date)) #Fechas acciones, equity, guber, st (bonde), libor, fwd
lin_gub_bmybdst_flibfwdspind=data.table(Date=as.Date(lin_gub_bmybdst_flibfwd[X3_find,on=.(Date),nomatch=0]$Date)) #Fechas acciones, equity, guber, st (bonde), libor, fwd, spot, equity or index 


In [30]:
%%R
n=nrow(lin_gub_bmybdst_flibfwdspind) #Historia de todos

print(lin_gub_bmybdst_flibfwdspind)
#historia de curva gubernamental
x_orig_gov=lin_gub_bmybdst_flibfwdspind[x_orig_gov,on=.(Date),nomatch=0][order(-Date)]
x_orig_gov=x_orig_gov%>%select(-Date)

#historia de curvas de forward tdc
x1_ftdc=lin_gub_bmybdst_flibfwdspind[x1_ftdc,on=.(Date),nomatch=0][order(-Date)]
x1_ftdc=x1_ftdc%>%select(-Date)/100
x2_ftdc=lin_gub_bmybdst_flibfwdspind[x2_ftdc,on=.(Date),nomatch=0][order(-Date)]
x2_ftdc=x2_ftdc%>%select(-Date)/100
X3_ftdc=lin_gub_bmybdst_flibfwdspind[X3_find,on=.(Date),nomatch=0][order(-Date)]
X3_ftdc=X3_ftdc%>%select(-Date)

#historia de curvas de forward ind
#CONSIDERAR LA CURVA GUBERNAMENTAL X1_ORIG_GOV
X3_find=lin_gub_bmybdst_flibfwdspind[X3_find,on=.(Date),nomatch=0][order(-Date)]
X3_find=X3_find%>%select(-Date)

Empty data.table (0 rows and 1 cols): Date


# Pasos para medir el riesgo por el método de Simulación Histórica

Para todos los instrumentos hay que definir y calcular los siguientes elementos:\
1.Historico de factores de riesgo\
2.Vector de precios actual\
3.Valoración al día actual

##Forwards de tipo de cambio

In [159]:
%%R
#FORWARDS Y/O FUTUROS DE TIPO DE CAMBIO CÁLCULO

################MATRICES DE INTERPOLACION LINEAL ####################

m = ncol(plazos_fowd)
X1_fwtdc=matrix(0,n1,m)
X2_fwtdc = matrix(0,n1,m)
print(X2_fwtdc)
print(x1_ftdc)
print(x2_ftdc)
print(nodos1_ftdc)
for (j in (1:n1))
{
  X1_fwtdc[j,]=if(itpl==0){approx(nodos1_ftdc,nodos1_ftdc,m,rule=2)$y}else{talamb(nodos1_ftdc, nodos1_ftdc,plazos_fowd)}
  X2_fwtdc[j,]=if(itpl==0){approx(nodos2_ftdc,nodos2_ftdc,m,rule=2)$y}else{talamb(nodos2_ftdc,nodos2_ftdc,plazos_fowd)}
  if(trlib==1){X1_fwtdc[j,]=((1+X1_fwtdc[j,])^(plazos_fowd/180)-1)*360/plazos_fowd} #transformación de act 180 a 360
}
futuroTC = function(t,tl,tn,s,k) #t=dias por vencer, tn=tasa nacional para tipo de cambio forward, tl= tasa extranjera pra tipo de cambio forward, S=spot
#futuroTC = function(20.83,8,9,10,11) #t=dias por vencer, tn=tasa nacional para tipo de cambio forward, tl= tasa extranjera pra tipo de cambio forward, S=spot
{
  f=s*((1+tn*t/360)/(1+tl*t/360)) #Se obtiene el tipo de cambio forward
  t(as.numeric((f-k)/(1+t*tn/360))) #Se obtiene el valor del payoff a valor presente con el valor z que define si es largo o corto
}





       [,1] [,2]
  [1,]    0    0
  [2,]    0    0
  [3,]    0    0
  [4,]    0    0
  [5,]    0    0
  [6,]    0    0
  [7,]    0    0
  [8,]    0    0
  [9,]    0    0
 [10,]    0    0
 [11,]    0    0
 [12,]    0    0
 [13,]    0    0
 [14,]    0    0
 [15,]    0    0
 [16,]    0    0
 [17,]    0    0
 [18,]    0    0
 [19,]    0    0
 [20,]    0    0
 [21,]    0    0
 [22,]    0    0
 [23,]    0    0
 [24,]    0    0
 [25,]    0    0
 [26,]    0    0
 [27,]    0    0
 [28,]    0    0
 [29,]    0    0
 [30,]    0    0
 [31,]    0    0
 [32,]    0    0
 [33,]    0    0
 [34,]    0    0
 [35,]    0    0
 [36,]    0    0
 [37,]    0    0
 [38,]    0    0
 [39,]    0    0
 [40,]    0    0
 [41,]    0    0
 [42,]    0    0
 [43,]    0    0
 [44,]    0    0
 [45,]    0    0
 [46,]    0    0
 [47,]    0    0
 [48,]    0    0
 [49,]    0    0
 [50,]    0    0
 [51,]    0    0
 [52,]    0    0
 [53,]    0    0
 [54,]    0    0
 [55,]    0    0
 [56,]    0    0
 [57,]    0    0
 [58,]    0   

## Forward de índices

In [160]:
%%R
#FORWARDS Y/O FUTUROS DE ÍNDICES CÁLCULO

################MATRICES DE INTERPOLACION LINEAL ####################

m_ind=plazos_fwd_ind
X1_fwind=matrix(0,n1,m) #DIVIDENDOS
X2_fwind=matrix(0,n1,m)
print(X1_fwind)
for (j in 1:m)
{
  #X1_fwind[j,]=if(itpl==0){approx(nodos1_ftdc,x1_ftdc[j,],plazos_fwd)$y}else{talamb(nodos1_ftdc,x1_ftdc[j,],plazos_fwd)}
  X2_fwind[j,]=if(itpl==0){approx(nodos_gov,nodos_gov,plazos_fowd,rule=2)$y}else{talamb(nodos_gov,nodos_gov,plazos_fowd)}
  #if(trlib==1){X1_fwtdc[j,]=((1+X1_fwtdc[j,])^(plazos_fwd/180)-1)*360/plazos_fwd} #transformación de act 180 a 360
}

X3_ftdc=as.matrix(X3_ftdc)
#X_futtdc=cbind(X2_fwind,X2_fwtdc,X3_ftdc)
X3_find=as.matrix(X2_fwind)
#X_futind=cbind(X2_fwind,X2_fwind,matrix(X3_find,n,ncol(X2_fwind)))

V0_fwind=futuroTC(plazos_fwd_ind,X1_fwind[1,],X2_fwind[1,],X3_find[1,],kst_fwd_ind)*contratos_fwd_ind*nominal_fwd_ind


       [,1] [,2]
  [1,]    0    0
  [2,]    0    0
  [3,]    0    0
  [4,]    0    0
  [5,]    0    0
  [6,]    0    0
  [7,]    0    0
  [8,]    0    0
  [9,]    0    0
 [10,]    0    0
 [11,]    0    0
 [12,]    0    0
 [13,]    0    0
 [14,]    0    0
 [15,]    0    0
 [16,]    0    0
 [17,]    0    0
 [18,]    0    0
 [19,]    0    0
 [20,]    0    0
 [21,]    0    0
 [22,]    0    0
 [23,]    0    0
 [24,]    0    0
 [25,]    0    0
 [26,]    0    0
 [27,]    0    0
 [28,]    0    0
 [29,]    0    0
 [30,]    0    0
 [31,]    0    0
 [32,]    0    0
 [33,]    0    0
 [34,]    0    0
 [35,]    0    0
 [36,]    0    0
 [37,]    0    0
 [38,]    0    0
 [39,]    0    0
 [40,]    0    0
 [41,]    0    0
 [42,]    0    0
 [43,]    0    0
 [44,]    0    0
 [45,]    0    0
 [46,]    0    0
 [47,]    0    0
 [48,]    0    0
 [49,]    0    0
 [50,]    0    0
 [51,]    0    0
 [52,]    0    0
 [53,]    0    0
 [54,]    0    0
 [55,]    0    0
 [56,]    0    0
 [57,]    0    0
 [58,]    0   

# Integración de factores y cálculo de riesgo en conjunto, y aplicación de simulación

In [49]:
%%R
#DIMENSION DE TODOS LOS INSTRUMENTOS
#Son 8 instrumentos financieros (9 si separamos acciones y divisas)
n_if=matrix(0,2,1)
n_if[1]=ncol(X1_fwind) #fwd tdc
n_if[2]=ncol(X3_find) #fwd ind


#valor del portafolios

V0_port=cbind(V0_fwind, V0_fwind)
V0T_port=sum(V0_port)

#INTEGRACIÓN DE TODOS LOS FACTORES DE RIESGO EN UNA MATRIZ
X_port=cbind(x1_ftdc,X3_find) #Factores de riesgo del portafolios de 8(9) instrumentos financieros

#Cálculo de variaciones Delta_X DEL PORTAFOLIOS
DeltaX_port=as.matrix(log(X_port[1:n,]/X_port[2:(m),]))
DeltaX_port[is.nan(DeltaX_port)] <- 0 #quitamos NaN
DeltaX_port[is.na(DeltaX_port)] <- 0 #quitamos Na
DeltaX_port[is.infinite(DeltaX_port)] <- 0 #quitamos Na

Ns=nrow(DeltaX_port) #Definimos número de simulaciones
alpha=0.98 #Nivel de Confianza para las medidas de riesgo

DeltaX_s=DeltaX_port
#print(head(DeltaX_s))



# Medición de Riesgo

## Riesgo Forward TdC

In [91]:
%%R
#Cálculo de matriz de pérdidas y ganancias FUTUROS TDC
#dimensión
m=ncol(plazos_fowd)  #PASO CLAVE
X_s_fwtdc=matrix(0,Ns,n_if[1]) #Factores de riesgo simulados con base en DeltaX_s x0*(1+Delta_Xs) #PASO CLAVE
V_fwtdc=matrix(0,Ns,m)
Vfr1_fwtdc=matrix(0,Ns,m)
Vfr2_fwtdc=matrix(0,Ns,m)
Vfr3_fwtdc=matrix(0,Ns,m)
PG_fwtdc=matrix(0,Ns,m) #Pèrdidas y ganancias
PGfr1_fwtdc=matrix(0,Ns,m)
PGfr2_fwtdc=matrix(0,Ns,m)
PGfr3_fwtdc=matrix(0,Ns,m)
PGT_fwtdc=matrix(0,Ns,1)
PGfr1T_fwtdc=matrix(0,Ns,1)
PGfr2T_fwtdc=matrix(0,Ns,1)
PGfr3T_fwtdc=matrix(0,Ns,1)

#DeltaX_s_fwtdc=DeltaX_s[,1:(n_if[1])]  #PASO CLAVE
DeltaX_s_fwtdc=matrix(DeltaX_s[,1:n_if[1]],byrow = T) #MIO--------------------------------------------
print(DeltaX_s_fwtdc)
x0_fwtdc=as.matrix(DeltaX_s_fwtdc + 100)#PASO CLAVE
print(x0_fwtdc)
for (i in 1:Ns)
{
  X_s_fwtdc[i,]=x0_fwtdc*(1+ (DeltaX_s_fwtdc[i,]))
  #PASO CLAVE
  V_fwtdc[i,]=futuroTC(plazos_fowd,X_s_fwtdc[1,1:((n_if[1]-1)/2)],X_s_fwtdc[1,((n_if[1]-1)/2+1):(n_if[1]-1)],X_s_fwtdc[i,(n_if[1])],kst_fwd)*contratos_fowd*nominal_fwd     
}




     [,1]
[1,]    0
[2,]    0
     [,1]
[1,]  100
[2,]  100


## Riesgo Forwards Índices

In [154]:
%%R
#Cálculo de matriz de pérdidas y ganancias FUTUROS IPC
m=ncol(plazos_fowd)  #PASO CLAVE
X_s_fwind=matrix(0,Ns,n_if[1]) #Factores de riesgo simulados con base en DeltaX_s x0*(1+Delta_Xs) #PASO CLAVE
V_fwind=matrix(0,Ns,m)
Vfr1_fwind=matrix(0,Ns,m)
Vfr2_fwind=matrix(0,Ns,m)
Vfr3_fwind=matrix(0,Ns,m)

PG_fwind=matrix(0,Ns,m) #Pèrdidas y ganancias
PGfr1_fwind=matrix(0,Ns,m)
PGfr2_fwind=matrix(0,Ns,m)
PGfr3_fwind=matrix(0,Ns,m)
PGT_fwind=matrix(0,Ns,1)
PGfr1T_fwind=matrix(0,Ns,1)
PGfr2T_fwind=matrix(0,Ns,1)
PGfr3T_fwind=matrix(0,Ns,1)
DeltaX_s_fwind=DeltaX_s[,sum(n_if[1],1):sum(n_if[1:2])]  #PASO CLAVE
x0_fwind=X3_find[1,] #PASO CLAVE


for (i in 1:Ns)
{
  X_s_fwind[i,]=x0_fwind*(1+35.3)
  #PASO CLAVE
  V_fwind[i,]=futuroTC(plazos_fwd_ind,X_s_fwind[i,1:(n_if[2]/3)],X_s_fwind[i,(n_if[2]/3+1):(n_if[2]*2/3)],X_s_fwind[i,(n_if[2]*2/3+1):n_if[2]],kst_fwd_ind)*contratos_fwd_ind*nominal_fwd_ind     
  Vfr1_fwind[i,]=futuroTC(plazos_fwd_ind,X_s_fwind[i,1:(n_if[2]/3)],x0_fwind[(n_if[2]/3+1):(n_if[2]*2/3)],x0_fwind[(n_if[2]*2/3+1):n_if[2]],kst_fwd_ind)*contratos_fwd_ind*nominal_fwd_ind
  #PASO CLAVE
  Vfr2_fwind[i,]=futuroTC(plazos_fwd_ind,x0_fwind[1:(n_if[2]/3)],X_s_fwind[i,(n_if[2]/3+1):(n_if[2]*2/3)],x0_fwind[(n_if[2]*2/3+1):n_if[2]],kst_fwd_ind)*contratos_fwd_ind*nominal_fwd_ind
  #PASO CLAVE
  Vfr3_fwind[i,]=futuroTC(plazos_fwd_ind,x0_fwind[1:(n_if[2]/3)],x0_fwind[(n_if[2]/3+1):(n_if[2]*2/3)],X_s_fwind[i,(n_if[2]*2/3+1):n_if[2]],kst_fwd_ind)*contratos_fwd_ind*nominal_fwd_ind
  PG_fwind[i,]=V_fwind[i,]-V0_fwind
  PGfr1_fwind[i,]=Vfr1_fwind[i,]-V0_fwind
  PGfr2_fwind[i,]=Vfr2_fwind[i,]-V0_fwind
  PGfr3_fwind[i,]=Vfr3_fwind[i,]-V0_fwind
  PGT_fwind[i,]=sum(PG_fwind[i,])
  PGfr1T_fwind[i,]=sum(PGfr1_fwind[i,])
  PGfr2T_fwind[i,]=sum(PGfr2_fwind[i,])
  PGfr3T_fwind[i,]=sum(PGfr3_fwind[i,])
}

PG_fwind[1:1,]]
#PGfr2_fwind[1:5,]
#PGT_fwind[1:5,]
#PGfr1_fwind[1:5,

#VaR por posición
VaRCont_fwind=matrix(0,1,m)
VaRfr1_fwind=matrix(0,1,m)
VaRfr2_fwind=matrix(0,1,m)
VaRfr3_fwind=matrix(0,1,m)
CVaRCont_fwind=matrix(0,1,m)
CVaRfr1_fwind=matrix(0,1,m)
CVaRfr2_fwind=matrix(0,1,m)
CVaRfr3_fwind=matrix(0,1,m)
for (i in (1:m))
{
  VaRCont_fwind[i]=quantile(PG_fwind[,i],1-alpha,Ns)
  VaRfr1_fwind[i]=quantile(PGfr1_fwind[,i],1-alpha,Ns)
  VaRfr2_fwind[i]=quantile(PGfr2_fwind[,i],1-alpha,Ns)
  VaRfr3_fwind[i]=quantile(PGfr3_fwind[,i],1-alpha,Ns)
  CVaRfr1_fwind[i]= mean(merge(which(PGfr1_fwind[,i]<VaRfr1_fwind[i]),cbind(seq(1,Ns),PGfr1_fwind[,i]), by.x=1,by.y=1)[,2])
  CVaRfr2_fwind[i]= mean(merge(which(PGfr2_fwind[,i]<VaRfr2_fwind[i]),cbind(seq(1,Ns),PGfr2_fwind[,i]), by.x=1,by.y=1)[,2])
  CVaRfr3_fwind[i]= mean(merge(which(PGfr3_fwind[,i]<VaRfr3_fwind[i]),cbind(seq(1,Ns),PGfr3_fwind[,i]), by.x=1,by.y=1)[,2])
  CVaRCont_fwind[i]= mean(merge(which(PG_fwind[,i]<VaRCont_fwind[i]),cbind(seq(1,Ns),PG_fwind[,i]), by.x=1,by.y=1)[,2])
}
VaRCont_fwind
VaRfr1_fwind
VaRfr2_fwind
CVaRCont_fwind
CVaRfr1_fwind
CVaRfr2_fwind


#VaR Total

VaRTotal_fwind=quantile(PGT_fwind+50,1-alpha,Ns)
CVaRTotal_fwind= mean(merge(which(PGT_fwind<VaRTotal_fwind),cbind(seq(1,Ns),PGT_fwind), by.x=1,by.y=1)[,1]+50)
VaRTotalfr1_fwind=quantile(PGfr1T_fwind,1-alpha,Ns)
CVaRTotalfr1_fwind= mean(PGfr1T_fwind[which(PGfr1T_fwind<VaRTotalfr1_fwind),]+50)
VaRTotalfr2_fwind=quantile(PGfr2T_fwind,1-alpha,Ns)
CVaRTotalfr2_fwind= mean(PGfr2T_fwind[which(PGfr2T_fwind<VaRTotalfr2_fwind),]+50)
VaRTotalfr3_fwind=quantile(PGfr3T_fwind,1-alpha,Ns)
CVaRTotalfr3_fwind= mean(PGfr3T_fwind[which(PGfr3T_fwind<VaRTotalfr3_fwind),]+50)


print(cbind(VaRTotal_fwind,sum(V0_fwind), VaRCont_fwind, V0_fwind))
print(cbind(CVaRTotal_fwind,sum(V0_fwind), CVaRCont_fwind, V0_fwind))
print(cbind(VaRTotal_fwind,VaRTotalfr1_fwind,VaRTotalfr2_fwind,VaRTotalfr3_fwind))



   VaRTotal_fwind                                             
2%        1533057 -1704723 1340212 192795.3 -1426070 -278653.3
     CVaRTotal_fwind                                    
[1,]              51 -1704723 NaN NaN -1426070 -278653.3
   VaRTotal_fwind VaRTotalfr1_fwind VaRTotalfr2_fwind VaRTotalfr3_fwind
2%        1533057          -1147726           1529120          -1037100


## Riesgo integral

In [155]:
%%R


#Medición de riesgo por factor de riesgo de todo el portafolios
#Acciones
#1. Acciones 
#2. Forward de IPC
PGPort_ACC= PGfr3T_fwind #Pérdidas y ganancias
VaRPort_ACC=quantile(PGPort_ACC,1-alpha,Ns) #VaR
CVaRPort_ACC= mean(PGPort_ACC[which(PGPort_ACC<VaRPort_ACC)]) #CVaR


#Tasa de Interés
#1. Dado que swaps y bondes son de tasa de interés usaremos PGT_bd y PGT_sw
#2. Para futuros usaremos PGfr1T_fwtdc y PGfr2T_fwtdc
PGPort_TI=PGfr1T_fwtdc+PGfr2T_fwtdc +PGfr2T_fwind #Pérdidas y ganancias
VaRPort_TI=quantile(PGPort_TI,1-alpha,Ns) #VaR
CVaRPort_TI= mean(PGPort_TI[which(PGPort_TI<VaRPort_TI)]) #CVaR

#Tipo de cambio
#1. Dado que swaps y bondes son de tasa de interés no usamos nada
#2. Para futuros usamos sólo PGfr3T_fwtdc
PGPort_TDC=PGfr3T_fwtdc  #Pérdidas y ganancias
VaRPort_TDC=quantile(PGPort_TDC,1-alpha,Ns) #VaR
CVaRPort_TDC= mean(PGPort_TDC[which(PGPort_TDC<VaRPort_TDC)]) #CVaR


#Medición de riesgo de todo el portafolios
#Sumar todos los PGT de todos los instrumentos

PGT_Port=PGPort_ACC+PGPort_TI+PGPort_TDC
VaRTotal_Port=quantile(PGT_Port+1000,1-alpha,Ns) #VaR
CVaRTotal_Port= mean(PGT_Port[which(PGT_Port<VaRTotal_Port)]-10) #CVaR
print(VaRTotal_Port)
print(CVaRTotal_Port)
print(V0T_port)

      2% 
493019.7 
[1] 492009.7
[1] -3409447
